In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import time
import random
import pandas as pd
import re

### 크롬 오픈 및 로그인 코드

In [27]:
# 크롬 오픈
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(5)

# 틱톡 오픈
url = 'https://www.tiktok.com/ko-KR'
driver.get(url)
driver.implicitly_wait(5)

# 파서 지정
page = driver.page_source # page_source : selenium으로 불러온 페이지의 html코드 저장
parser = BeautifulSoup(page,'html.parser')
#parser

# 로그인 버튼 클릭
#driver.find_element(by='xpath', value='//*[@id="header-login-button"]/div').click()

# 전화/이메일/아이디 사용 버튼 클릭
#driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div/div/div/div[2]/div[2]').click()

# 이메일 또는 아이디로 로그인 클릭
#driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div/form/div[1]/a').click()

# 이메일 입력
#input_email = driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/div[1]/input')
#input_email.send_keys('이메일')

# 비밀번호 입력
#input_password = driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/div[2]/div/input')
#input_password.send_keys('비밀번호')

# 로그인버튼 클릭
#driver.find_element(by='xpath', value='//*[@id="loginContainer"]/div[2]/form/button').click() #추후 돌아가는지 확인


### 틱톡커 리스트

In [28]:
### 틱톡커 리스트
list = ['jeracoolio','mydelicate','jeb_bi','ayheyt3u','luv2jna','olafflee','ashlynbeauty_','emchu_','__.annie.s','bwnniewony',
        'beautyuliya','_luvwonsnnx','onlydahia','rosiehashs','putrinavisyy','xaiyva','chueog._','noteswithlu','4chriisty',
        'pyum.pyum','lov3lyk7','cocoomiii','dearwonii_','vailnessi','izzydiaries','ririsglow_','songofskin','deesukii',
        'velcroluv','kwyeona','444.jennie','seda.eks','heartsfor.kbeauty','jookstogo','stylekorean_official'
        ,'.blissdiaries','imseah_','misscasxie',
        '28leeny','minxngyn','dearlychu8','luvv.tine','j.uuuuudy','ddalgimiilk','onlyrilee','chxrry.glowy','ddalqiwee','sammixtang','aeiezo.pdf','caspertheghostyy'
        ]

In [26]:
# 틱톡커 명수 : 50
len(list)

50

### 틱톡커 50명 : 팔로워수, 조회수, 틱톡커 이름

In [29]:
# 틱톡커 프로필 url for문
tiktoker_list = list

view_final_list = []
follower_final_list = []
name_final_list = []

for tiktoker_name in tiktoker_list:
    url2 = 'https://www.tiktok.com/@'+tiktoker_name
    driver.get(url2)
    driver.implicitly_wait(30)

    # 고정됨 영상의 개수 추출
    try:
        badges = driver.find_elements(By.CSS_SELECTOR, value='div[data-e2e="video-card-badge"]')
        badges_num = len(badges)
        print(tiktoker_name,'고정된 영상 개수:',badges_num)
    except Exception as e:
        badges_num = 0
        print(tiktoker_name,'고정된 영상 없음')
    
    # 조회수 추출
    for i in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_UP) # 위로 살짝 올렸다
        driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN) # 아래로 내려야 다음 게시물들이 로딩된다...ㅠㅠ
        driver.implicitly_wait(5)

    # 잠시 멈춤 
    driver.implicitly_wait(10)

    views = driver.find_elements(By.CSS_SELECTOR, 'strong[data-e2e="video-views"]')
    views_total_list = []
    for view in views:
        view_text = view.text
        views_total_list.append(view_text)

    desired_view_cnt = 30 + badges_num

    if len(views_total_list) >= desired_view_cnt:

        if badges == 0:
            view_list = views_total_list[:30]
        else:
            del views_total_list[:badges_num]
            view_list = views_total_list[:30]
    else:
        if badges == 0:
            view_list = views_total_list
        else:
            del views_total_list[:badges_num]
            view_list = views_total_list

    print(tiktoker_name,'게시물 개수:',len(view_list))
    print(tiktoker_name,'조회수:',view_list)
    view_final_list.extend(view_list) #틱톡커 1명분 리스트를 전체 리스트와 합침

    # 잠시 멈춤
    driver.implicitly_wait(10)

    # 팔로워수 추출
    follower = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="followers-count"]'))
    )
    follower_text = follower.text

    follower_list = []
    for i in range(len(view_list)):
        follower_list.append(follower_text)
    
    print(tiktoker_name,'팔로워수:',follower_list)
    follower_final_list.extend(follower_list)

    # 틱톡커 이름 추출
    name_list = []
    for i in range(len(view_list)):
        name_list.append(tiktoker_name)
    
    print(tiktoker_name,'이름:',name_list)
    name_final_list.extend(name_list)

print('추출 완료했습니다.')
print('total 조회수:',view_final_list)
print('total 팔로워수:',follower_final_list)
print('total 틱톡커 이름:',name_final_list)

jeracoolio 고정된 영상 개수: 1
jeracoolio 게시물 개수: 30
jeracoolio 조회수: ['5361', '2407', '8390', '3329', '13.3K', '1966', '70.7K', '57.8K', '1964', '1974', '1491', '1568', '1239', '1775', '2282', '1891', '2250', '1600', '2467', '2130', '2232', '2259', '2392', '1385', '2347', '2559', '5308', '2358', '9403', '2804']
jeracoolio 팔로워수: ['5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482', '5482']
jeracoolio 이름: ['jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio', 'jeracoolio']


In [30]:
print(len(view_final_list))
print(len(follower_final_list))
print(len(name_final_list))

1500
1500
1500


In [31]:
df1 = pd.DataFrame({'name':name_final_list,
                    'follower':follower_final_list,
                    'view':view_final_list})

In [32]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      1500 non-null   object
 1   follower  1500 non-null   object
 2   view      1500 non-null   object
dtypes: object(3)
memory usage: 35.3+ KB


In [33]:
df1.to_csv("tiktoker_df1_0121.csv", index=False, encoding="utf-8-sig")

### 틱톡커 50명 : 좋아요수, 댓글수, 저장수, 업로드날짜, 영상설명, 해시태그

In [20]:
# 틱톡커 프로필 url for문
tiktoker_list = list2

like_final_list = []
comment_final_list = []
save_final_list = []
date_final_list = []
infos_tags_final_list = []
hash_tag_final_list = []

for tiktoker_name in tiktoker_list:
    url2 = 'https://www.tiktok.com/@'+tiktoker_name
    driver.get(url2)
    driver.implicitly_wait(30)

    # 고정됨 영상의 개수 추출
    try:
        badges = driver.find_elements(By.CSS_SELECTOR, value='div[data-e2e="video-card-badge"]')
        badges_num = len(badges)
        print(tiktoker_name,'고정된 영상 개수:',badges_num)
    except Exception as e:
        badges_num = 0
        print(tiktoker_name,'고정된 영상 없음')
    
    # 첫번 째 동영상 클릭
    driver.find_element(By.CSS_SELECTOR, 'div[data-e2e="user-post-item"]').click()

    # 고정된 영상 여부에 따른 정보 추출
    n = 30
    if badges_num == 0: # 고정된 영상이 없는 경우 
        
        like_list = []
        comment_list = []
        save_list = []
        infos_tags_list = []
        hash_tag_list = []
        date_list = []

        for i in range(n):
            # 자세히 버튼 존재하면 클릭
            try:
                expand_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'css-1r94cis-ButtonExpand'))
                )
                expand_button.click()
                print(i,'번:',"자세히 버튼 클릭 성공")
            except Exception as e:
                # 버튼이 없거나 클릭할 수 없을 경우 처리
                print(i,'번:',"자세히 버튼이 없거나 클릭할 수 없습니다.")
            
            # 영상설명 추출
            try:
                infos_tags = driver.find_element(By.CSS_SELECTOR, value = 'h1[data-e2e="browse-video-desc"]')
                infos_tags_text = infos_tags.text
                infos_tags_list.append(infos_tags_text)
            except NoSuchElementException:
                infos_tags_list.append('')

            # 해시태그 추출 
            hash_tags = driver.find_elements(By.CSS_SELECTOR, value = 'a.css-sbcvet-StyledCommonLink.ejg0rhn6.link-a11y-focus strong')
            hash_tag_text_list = []

            for hash_tag in hash_tags: # 추출된 요소의 텍스트 출력
                hash_tag_text = hash_tag.text.strip()

                if hash_tag_text:
                    hash_tag_text_list.append(hash_tag_text)
                else:
                    continue
            hash_tag_list.append(hash_tag_text_list)

            # 잠시 멈춤
            driver.implicitly_wait(10)

            #업로드 날짜 추출
            try:
                date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
                    )
                
                # 잠시 멈춤
                driver.implicitly_wait(5)

                date_text = date.text
                date_text = date_text.split('·')[1].strip()
                date_list.append(date_text)

            except StaleElementReferenceException:
                print("StaleElementReferenceException 발생: 요소가 더 이상 유효하지 않습니다.")
                date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
                    )
                
                # 잠시 멈춤
                driver.implicitly_wait(5)

                date_text = date.text
                date_text = date_text.split('·')[1].strip()
                date_list.append(date_text)
            
            except TimeoutException:
                print("업로드 날짜를 찾을 수 없습니다.")
                
            # 좋아요수 추출
            like = WebDriverWait(driver,10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-like-count"]'))
            )
            like_text = like[0].text
            like_list.append(like_text)

            # 댓글수 추출
            comment = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-comment-count"]'))
            )
            comment_text = comment.text
            comment_list.append(comment_text)

            # 저장수 추출
            save = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="undefined-count"]'))
            )
            save_text = save.text
            save_list.append(save_text)

            # 잠시 멈춤
            driver.implicitly_wait(3)
            
            # 다음 영상으로 넘어가기
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN)

        print(tiktoker_name,'영상 개수:',len(date_list))
        print(tiktoker_name,'좋아요:',like_list)
        print(tiktoker_name,'댓글수:',comment_list)
        print(tiktoker_name,'저장수:',save_list)
        print(tiktoker_name,'업로드 날짜:',date_list)
        print(tiktoker_name,'영상설명 및 해시태그:',date_list)
        print(tiktoker_name,'해시태그:',hash_tag_list)

        like_final_list.extend(like_list)
        comment_final_list.extend(comment_list)
        save_final_list.extend(save_list)
        date_final_list.extend(date_list)
        infos_tags_final_list.extend(infos_tags_list)
        hash_tag_final_list.extend(hash_tag_list)

    else: #고정된 영상이 있는 경우
        for i in range(badges_num):
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN) # 고정된 영상 개수만큼 넘기기
        
        like_list = []
        comment_list =[]
        save_list = []
        infos_tags_list = []
        hash_tag_list = []
        date_list = []
        
        for i in range(n):

            try:
                expand_button = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'css-1r94cis-ButtonExpand'))
                )
                expand_button.click()
                print(i,'번:',"자세히 버튼 클릭 성공")
            except Exception as e:
                # 버튼이 없거나 클릭할 수 없을 경우 처리
                print(i,'번:',"자세히 버튼이 없거나 클릭할 수 없습니다.")
            
            driver.implicitly_wait(5)

            # 영상설명 추출
            try:
                infos_tags = driver.find_element(By.CSS_SELECTOR, value = 'h1[data-e2e="browse-video-desc"]')
                infos_tags_text = infos_tags.text
                infos_tags_list.append(infos_tags_text)
            except NoSuchElementException:
                infos_tags_list.append('')

            # 해시태그 추출 
            hash_tags = driver.find_elements(By.CSS_SELECTOR, value = 'a.css-sbcvet-StyledCommonLink.ejg0rhn6.link-a11y-focus strong')
            hash_tag_text_list = []

            for hash_tag in hash_tags: # 추출된 요소의 텍스트 출력
                hash_tag_text = hash_tag.text.strip()

                if hash_tag_text:
                    hash_tag_text_list.append(hash_tag_text)
                else:
                    continue
            hash_tag_list.append(hash_tag_text_list)

            # 잠시 멈춤
            driver.implicitly_wait(10)

            try:
                date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
                    )
                
                # 잠시 멈춤
                driver.implicitly_wait(5)

                date_text = date.text
                date_text = date_text.split('·')[1].strip()
                date_list.append(date_text)

            except StaleElementReferenceException:
                print("StaleElementReferenceException 발생: 요소가 더 이상 유효하지 않습니다.")
                date = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.css-gg0x0w-SpanOtherInfos.evv7pft3'))
                    )
                
                # 잠시 멈춤
                driver.implicitly_wait(5)

                date_text = date.text
                date_text = date_text.split('·')[1].strip()
                date_list.append(date_text)
            
            except TimeoutException:
                print("업로드 날짜를 찾을 수 없습니다.")

            # 좋아요수 추출
            like = WebDriverWait(driver,10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-like-count"]'))
            )
            like_text = like[0].text
            like_list.append(like_text)

            # 댓글수 추출
            comment = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="browse-comment-count"]'))
            )
            comment_text = comment.text
            comment_list.append(comment_text)

            # 저장수 추출
            save = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'strong[data-e2e="undefined-count"]'))
            )
            save_text = save.text
            save_list.append(save_text)

            # 잠시 멈춤
            driver.implicitly_wait(5)

            # 다음 영상으로 넘어가기
            driver.find_element(By.TAG_NAME,'body').send_keys(Keys.ARROW_DOWN)
        
        print(tiktoker_name,'영상 개수',len(date_list))
        print(tiktoker_name,'좋아요:',like_list)
        print(tiktoker_name,'댓글수:',comment_list)
        print(tiktoker_name,'저장수:',save_list)
        print(tiktoker_name,'업로드 날짜:',date_list)
        print(tiktoker_name,'영상정보 및 해시태그:',infos_tags_list)
        print(tiktoker_name,'해시태그:',hash_tag_list)

        like_final_list.extend(like_list)
        comment_final_list.extend(comment_list)
        save_final_list.extend(save_list)
        date_final_list.extend(date_list)
        infos_tags_final_list.extend(infos_tags_list)
        hash_tag_final_list.extend(hash_tag_list)

print('정보 수집 ㄲ으읏')
print('total 좋아요:', like_final_list)
print('total 댓글수:', comment_final_list)
print('total 저장수:', save_final_list)
print('total 업로드 날짜:', date_final_list)
print('total 영상정보 및 해시태그:', infos_tags_final_list)
print('total 해시태그:', hash_tag_final_list)

.blissdiaries 고정된 영상 개수: 3
0 번: 자세히 버튼 클릭 성공
1 번: 자세히 버튼 클릭 성공
2 번: 자세히 버튼 클릭 성공
3 번: 자세히 버튼 클릭 성공
4 번: 자세히 버튼 클릭 성공
5 번: 자세히 버튼 클릭 성공
6 번: 자세히 버튼 클릭 성공
7 번: 자세히 버튼 클릭 성공
8 번: 자세히 버튼 클릭 성공
9 번: 자세히 버튼 클릭 성공
10 번: 자세히 버튼 클릭 성공
11 번: 자세히 버튼 클릭 성공
12 번: 자세히 버튼 클릭 성공
13 번: 자세히 버튼 클릭 성공
14 번: 자세히 버튼 클릭 성공
15 번: 자세히 버튼 클릭 성공
16 번: 자세히 버튼 클릭 성공
17 번: 자세히 버튼 클릭 성공
18 번: 자세히 버튼 클릭 성공
19 번: 자세히 버튼 클릭 성공
20 번: 자세히 버튼 클릭 성공
21 번: 자세히 버튼 클릭 성공
22 번: 자세히 버튼 클릭 성공
23 번: 자세히 버튼 클릭 성공
StaleElementReferenceException 발생: 요소가 더 이상 유효하지 않습니다.
24 번: 자세히 버튼 클릭 성공
25 번: 자세히 버튼 클릭 성공
26 번: 자세히 버튼 클릭 성공
27 번: 자세히 버튼 클릭 성공
28 번: 자세히 버튼 클릭 성공
29 번: 자세히 버튼 클릭 성공
.blissdiaries 영상 개수 30
.blissdiaries 좋아요: ['1311', '437', '1628', '1330', '824', '1115', '1003', '2672', '1656', '463', '371', '680', '1613', '1362', '1063', '987', '974', '2051', '1561', '1712', '1432', '5382', '1827', '974', '2106', '2550', '2450', '923', '807', '4301']
.blissdiaries 댓글수: ['48', '32', '56', '72', '61', '82', '36', '99', '79', '33', '25',

In [21]:
print(len(like_final_list))
print(len(comment_final_list))
print(len(save_final_list))
print(len(date_final_list))
print(len(infos_tags_final_list))
print(len(hash_tag_final_list))

450
450
450
450
450
450


In [22]:
df2 = pd.DataFrame({'like':like_final_list,
                    'comment':comment_final_list,
                    'save':save_final_list,
                    'date':date_final_list,
                    'info_tag':infos_tags_final_list,
                    'hash_tag':hash_tag_final_list})

In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   like      450 non-null    object
 1   comment   450 non-null    object
 2   save      450 non-null    object
 3   date      450 non-null    object
 4   info_tag  450 non-null    object
 5   hash_tag  450 non-null    object
dtypes: object(6)
memory usage: 21.2+ KB


In [24]:
df2.to_csv('tiktoker_df3_0121.csv',index=False, encoding="utf-8-sig")

### 데이터프레임 합치기

In [35]:
# 데이터 불러오기
info_df1 = pd.read_csv('C:/Users/user/Desktop/databootcamp/tiktok_test/tiktoker_df2_0121.csv')
info_df2 = pd.read_csv('C:/Users/user/Desktop/databootcamp/tiktok_test/tiktoker_df3_0121.csv')

view_df = pd.read_csv('C:/Users/user/Desktop/databootcamp/tiktok_test/tiktoker_df1_0121.csv')

In [36]:
# info_df 세로 병합
info_concat_df = pd.concat([info_df1,info_df2])
info_concat_df

,Unnamed: 0,like,comment,save,date,info_tag,hash_tag
0,0,587,21,66,2024-9-24,What shade is your favorite?? 🤭\n\nive been us...,What shade is your favorite?? 🤭\n\nive been us...
1,1,249,17,34,2024-9-1,new 20% off code !!🤭 expires soon!!\n\n@YesSty...,new 20% off code !!🤭 expires soon!!\n\n@YesSty...
2,2,363,19,65,2024-9-1,obsessedddd with this serum🤞\n\nu can get it o...,obsessedddd with this serum🤞\n\nu can get it o...
3,3,275,358,47,2024-8-28,Good luckkkk💓💓\n! Winner will be dmed from thi...,Good luckkkk💓💓\n! Winner will be dmed from thi...
4,4,696,33,127,2024-8-27,Its suchhhh a good purchase!! saves u alot of ...,Its suchhhh a good purchase!! saves u alot of ...
...,...,...,...,...,...,...,...
445,445,8214,46,727,2024-12-26,Getting my glow up before entering 2025✨💇🏻‍♀️@...,Getting my glow up before entering 2025✨💇🏻‍♀️@...
446,446,3959,32,247,2024-12-25,Merry Christmas everyone🎄❄️✨#mixsoon #koreanof...,Merry Christmas everyone🎄❄️✨#mixsoon #koreanof...
447,447,2797,53,541,2024-12-24,I always go for @Veganifect_global gel cleanse...,I always go for @Veganifect_global gel cleanse...
448,448,1992,31,124,2024-12-23,Opening squid game 2 x bringgreen & wakemake b...,Opening squid game 2 x bringgreen & wakemake b...


In [40]:
# info_concat_df unnamed컬럼 버리기
info_concat_df = info_concat_df[['like', 'comment', 'save', 'date', 'info_tag','hash_tag']]

In [44]:
info_concat_df = info_concat_df.reset_index(drop=True)

In [45]:
view_df = view_df.reset_index(drop=True)

In [46]:
# info_concat_df, view_df 가로 병합
final_df = pd.concat([view_df,info_concat_df],axis=1)

In [48]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      1500 non-null   object
 1   follower  1500 non-null   object
 2   view      1500 non-null   object
 3   like      1500 non-null   object
 4   comment   1500 non-null   int64 
 5   save      1500 non-null   object
 6   date      1500 non-null   object
 7   info_tag  1492 non-null   object
 8   hash_tag  1494 non-null   object
dtypes: int64(1), object(8)
memory usage: 105.6+ KB


In [49]:
final_df.to_csv('tiktoker_crawling_df_0121.csv')